In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove

R0 Claiming a refund by an investor decreases the escrow’s balance by the investor’s deposit.

$
\varphi_{\mathbf{R} 0} \equiv \square \text { (claimRefund (address p) } \rightarrow \text { Escrow.balance } = \text { Escrow.balance }{\bullet} \text { Escrow.deposits }^{\bullet}[\mathrm{p}] \text { ) }
$

R1 The escrow’s balance must be at least the sum of investor deposits, unless the crowdsale is declared successful.

$
\varphi_{\mathbf{R} 1} \equiv \square(\text { state } \neq \text { success } \rightarrow \operatorname{sum}(\text { deposits) } \leq \text { Escrow.balance) }
$

R2 The escrow never allows the beneficiary to withdraw the investments and the investors to claim refunds.

$
\varphi_{\mathbf{R} \mathbf{2}} \equiv \square(\neg(\text { withdraw () } \wedge \text { claimRefund() }))
$

R3 Investors cannot claim refunds after more than 10, 000 ether is collected.

$
\left.\varphi_{\mathbf{R 3}} \equiv \square(\text { claimRefund() } \rightarrow \neg(\text { sum(deposits }) \geq \text { goal })\right)
$

In [2]:
# The smart contract as a transition system.
ts = Ts('Crowdsale')

GOAL = 10000
CLOSETIME = 10000

addrSort = z3.BitVecSort(256)

state, stateOut = ts.add_var(z3.BitVecSort(2), name='state')
OPEN = 0
SUCCESS = 1
REFUND = 2

deposits, depositsOut = ts.add_var(z3.ArraySort(addrSort, z3.BitVecSort(256)),
        name= 'deposits')

totalDeposits, totalDepositsOut = ts.add_var(z3.BitVecSort(256), name='totalDeposits')

raised, raisedOut = ts.add_var(z3.BitVecSort(256), name='raised')

aux_withdraw, aux_withdrawOut = ts.add_var(z3.BoolSort(), name='aux_withdraw')

aux_refund, aux_refundOut = ts.add_var(z3.BoolSort(), name='aux_refund')


func, funcOut = ts.add_var(z3.StringSort(), name='func')

timestamp, timestampOut = ts.add_var(z3.BitVecSort(256), name='now')

In [3]:
p = z3.BitVec('p',256)

ts.Init = z3.And(
    z3.ForAll(p, deposits[p]==0),
    raised == 0,
    totalDeposits == 0,
    aux_withdraw == False,
    aux_refund == False,
    state == OPEN,
    func == "init",
    timestamp == 0
)

amount = z3.BitVec('amount',256)

tr_invest = z3.And(
    # timestamp <= CLOSETIME,
    raised < GOAL,

    stateOut == state,
    raisedOut == raised + amount,
    depositsOut == z3.Update(deposits,p,deposits[p]+amount),
    totalDepositsOut == totalDeposits+amount,

    aux_refundOut == aux_refund,
    aux_withdrawOut == aux_withdraw,
    funcOut == "invest",
    timestampOut > timestamp
)

tr_close_success = z3.And(
    raised >= GOAL,

    depositsOut == deposits,
    raisedOut == raised,
    totalDepositsOut == totalDeposits,
    stateOut == SUCCESS,

    aux_refundOut == aux_refund,
    aux_withdrawOut == aux_withdraw,
    funcOut == "close_success",
    timestampOut > timestamp
)

tr_close_refund = z3.And(
    timestamp > CLOSETIME,
    raised < GOAL,

    stateOut == REFUND,
    depositsOut == deposits,
    raisedOut == raised,
    totalDepositsOut == totalDeposits,

    aux_refundOut == aux_refund,
    aux_withdrawOut == aux_withdraw,
    funcOut == "close_refund",
    timestampOut > timestamp
)

tr_claimrefund = z3.And(
    state == REFUND,
    raised < GOAL,

    depositsOut == z3.Update(deposits,p,0),
    totalDepositsOut == totalDeposits - deposits[p],
    raisedOut == raised,
    stateOut == state,

    aux_refundOut == True,
    aux_withdrawOut == aux_withdraw,
    funcOut == "claimrefund",
    timestampOut > timestamp
)

tr_withdraw = z3.And(
    state == SUCCESS,
    
    totalDepositsOut == 0,
    depositsOut == deposits,
    raisedOut == raised,
    stateOut == state,

    aux_withdrawOut == True,
    aux_refundOut == aux_refund,
    funcOut == "withdraw",
    timestampOut > timestamp
)

ts.Tr = z3.Or(tr_invest, tr_close_refund, tr_close_success, tr_claimrefund, tr_withdraw)

In [4]:
ts_property = raised == totalDeposits
r1 = z3.Implies(state == SUCCESS, raised >= GOAL+1)
r2 = z3.Not(z3.And(aux_withdraw, aux_refund))

s = prove_inductive(ts, r1)


Prove init => property.
proved
Prove property is inductive.
failed to prove:  sat
Implies(And(Implies(state == 1, raised >= 10001),
            Or(And(now <= 10000,
                   raised < 10000,
                   state' == state,
                   raised' == raised + amount,
                   deposits' ==
                   Store(deposits, p, deposits[p] + amount),
                   totalDeposits' == totalDeposits + amount,
                   aux_refund' == aux_refund,
                   aux_withdraw' == aux_withdraw,
                   func' == "invest",
                   now' > now),
               And(now > 10000,
                   raised < 10000,
                   state' == 2,
                   deposits' == deposits,
                   raised' == raised,
                   totalDeposits' == totalDeposits,
                   aux_refund' == aux_refund,
                   aux_withdraw' == aux_withdraw,
                   func' == "close_refund",
                   now' > 

In [5]:
from lib.bmc import *

In [6]:
g = z3.Not(r2)
fvs = [p, amount]
xs = [deposits, totalDeposits, raised, state, aux_withdraw, aux_refund, func, timestamp]
xns = [depositsOut, totalDepositsOut, raisedOut, stateOut, aux_withdrawOut, aux_refundOut, funcOut, timestampOut]
model = bmc(ts.Init, ts.Tr, g, fvs, xs, xns)
print(model)
extract_model(model, "func")
extract_model(model)

iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7


iteration  8
iteration  9
iteration  10
iteration  11
iteration  12
iteration  13
iteration  14
iteration  15
iteration  16
iteration  17
iteration  18
iteration  19
iteration  20
None


TypeError: 'NoneType' object is not iterable